In [ ]:
DATA_PATH = 'data/no_exogenous/data.csv'
FREQ = 'h'
YEAR = 2021
OUTPUT_PATH = f'data/predictions_mlp_{YEAR}.csv'

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

In [ ]:
data = pd.read_csv(DATA_PATH, parse_dates=['timestamp'])
data = data.set_index('timestamp').asfreq(FREQ).dropna()
data['year'] = data.index.year

train = data[(data['year'] < YEAR) & (data['year'] > YEAR - 5)]
test = data[data['year'] == YEAR]

X_train = train[['hour','day_of_week','month','year','is_weekend','quarter','is_holiday']]
y_train = train['value']
X_test = test[['hour','day_of_week','month','year','is_weekend','quarter','is_holiday']]

X_scaler = MinMaxScaler().fit(X_train)
y_scaler = MinMaxScaler().fit(y_train.values.reshape(-1,1))

X_train_s = X_scaler.transform(X_train)
y_train_s = y_scaler.transform(y_train.values.reshape(-1,1)).ravel()

In [ ]:
model = MLPRegressor(hidden_layer_sizes=(1024,512,128,32,8),
                     max_iter=500,
                     learning_rate='adaptive',
                     learning_rate_init=0.001,
                     tol=1e-6,
                     random_state=42)
model.fit(X_train_s, y_train_s)

In [ ]:
X_test_s = X_scaler.transform(X_test)
y_pred = y_scaler.inverse_transform(model.predict(X_test_s).reshape(-1,1)).flatten()


In [ ]:
out = test.copy()
out['value'] = y_pred
out.to_csv(OUTPUT_PATH, index=False)